# **Kompletny Tutorial: Docker LDAP + Python**

Ten tutorial krok po kroku pokaże, jak skonfigurować serwer LDAP za pomocą Dockera oraz jak pracować z nim w Pythonie.

---

## **Część 1: Uruchomienie LDAP za pomocą Dockera**

### **1.1 Pobranie i uruchomienie kontenera OpenLDAP**
Uruchom OpenLDAP w kontenerze Docker:


In [1]:

!docker run --name openldap -p 389:389 -p 636:636 -e LDAP_ORGANISATION="Example Inc." -e LDAP_DOMAIN="example.org" -e LDAP_ADMIN_PASSWORD="admin" -d osixia/openldap:latest


9ef308f05f746c3083e45d2d87a511ebb1f4f488c0d245faebaf3e32ac69bb51



- **LDAP_ORGANISATION**: Nazwa organizacji.
- **LDAP_DOMAIN**: Nazwa domeny.
- **LDAP_ADMIN_PASSWORD**: Hasło administratora LDAP.

### **1.2 Weryfikacja uruchomionego serwera**
Sprawdź, czy kontener działa:



In [2]:
!docker ps


CONTAINER ID   IMAGE                    COMMAND                 CREATED          STATUS          PORTS                                        NAMES
9ef308f05f74   osixia/openldap:latest   "/container/tool/run"   28 seconds ago   Up 27 seconds   0.0.0.0:389->389/tcp, 0.0.0.0:636->636/tcp   openldap




### **1.3 Test połączenia do serwera LDAP**
Użyj `ldapsearch`, aby sprawdzić konfigurację serwera:


In [3]:

!ldapsearch -x -H ldap://localhost -b "dc=example,dc=org" -D "cn=admin,dc=example,dc=org" -w admin "(objectClass=*)"


# extended LDIF
#
# LDAPv3
# base <dc=example,dc=org> with scope subtree
# filter: (objectClass=*)
# requesting: ALL
#

# example.org
dn: dc=example,dc=org
objectClass: top
objectClass: dcObject
objectClass: organization
o: Example Inc.
dc: example

# search result
search: 2
result: 0 Success

# numResponses: 2
# numEntries: 1




Jeśli wszystko działa poprawnie, zobaczysz wynik zawierający wpis `dc=example,dc=org`.

---

## **Część 2: Dodanie danych do LDAP**

### **2.1 Stwórz plik `users.ldif`**
Ten plik zawiera wpisy użytkowników i jednostek organizacyjnych:



In [5]:
%%writefile users.ldif
dn: ou=users,dc=example,dc=org
objectClass: organizationalUnit
ou: users

dn: uid=jdoe,ou=users,dc=example,dc=org
objectClass: inetOrgPerson
cn: John Doe
sn: Doe
uid: jdoe
mail: jdoe@example.org
userPassword: password


Writing users.ldif




### **2.2 Dodaj dane do LDAP**
Załaduj plik `users.ldif` do serwera LDAP:


In [6]:
!ldapadd -x -H ldap://localhost -D "cn=admin,dc=example,dc=org" -w admin -f users.ldif

adding new entry "ou=users,dc=example,dc=org"

adding new entry "uid=jdoe,ou=users,dc=example,dc=org"





---

## **Część 3: Praca z LDAP w Pythonie**

### **3.1 Instalacja python-ldap**
Zainstaluj bibliotekę:
```bash
pip install python-ldap
```



### **3.2 Tworzenie skryptu w Pythonie**
Stwórz plik `ldap_example.py` z poniższym kodem:


In [11]:
%%writefile ldap_example.py

import ldap

LDAP_SERVER = "ldap://localhost"
BASE_DN = "dc=example,dc=org"
ADMIN_DN = "cn=admin,dc=example,dc=org"
PASSWORD = "admin"

try:
    # Połączenie z serwerem LDAP
    conn = ldap.initialize(LDAP_SERVER)
    conn.simple_bind_s(ADMIN_DN, PASSWORD)
    print("Połączono z serwerem LDAP!")

    # Wyszukaj użytkowników
    search_filter = "(objectClass=inetOrgPerson)"
    search_attributes = ["cn", "sn", "mail"]
    results = conn.search_s(BASE_DN, ldap.SCOPE_SUBTREE, search_filter, search_attributes)

    # Wyświetl wyniki
    for dn, entry in results:
        print(f"DN: {dn}")
        for attr, values in entry.items():
            print(f"  {attr}: {values}")
except ldap.LDAPError as e:
    print(f"Błąd LDAP: {e}")
finally:
    conn.unbind()


Overwriting ldap_example.py




### **3.3 Uruchomienie skryptu**
Uruchom skrypt:


In [12]:
!pip install python-ldap

In [13]:
!python ldap_example.py

Połączono z serwerem LDAP!
DN: uid=jdoe,ou=users,dc=example,dc=org
  cn: [b'John Doe']
  sn: [b'Doe']
  mail: [b'jdoe@example.org']




Oczekiwany wynik:
```plaintext
Połączono z serwerem LDAP!
DN: uid=jdoe,ou=users,dc=example,dc=org
  cn: [b'John Doe']
  sn: [b'Doe']
  mail: [b'jdoe@example.org']
```

---



## **Część 4: Rozszerzone operacje LDAP w Pythonie**

### **4.1 Dodawanie nowych użytkowników**
Dodaj użytkownika za pomocą Pythona:

```python
new_user_dn = "uid=jane,ou=users,dc=example,dc=org"
attributes = {
    "objectClass": [b"inetOrgPerson"],
    "cn": [b"Jane Smith"],
    "sn": [b"Smith"],
    "uid": [b"jane"],
    "mail": [b"jane.smith@example.org"],
    "userPassword": [b"password"]
}

ldif = [(key, values) for key, values in attributes.items()]

try:
    conn.add_s(new_user_dn, ldif)
    print("Dodano nowego użytkownika!")
except ldap.LDAPError as e:
    print(f"Błąd dodawania: {e}")
```

### **4.2 Modyfikacja danych użytkownika**
Zmień adres e-mail użytkownika:
```python
user_dn = "uid=jdoe,ou=users,dc=example,dc=org"
modifications = [
    (ldap.MOD_REPLACE, "mail", [b"john.doe@newdomain.org"]),
]

try:
    conn.modify_s(user_dn, modifications)
    print("Zaktualizowano dane użytkownika!")
except ldap.LDAPError as e:
    print(f"Błąd modyfikacji: {e}")
```

### **4.3 Usuwanie użytkowników**
Usuń użytkownika:
```python
user_dn = "uid=jane,ou=users,dc=example,dc=org"

try:
    conn.delete_s(user_dn)
    print("Usunięto użytkownika!")
except ldap.LDAPError as e:
    print(f"Błąd usuwania: {e}")
```

---

## **Podsumowanie**
1. Uruchomiłeś serwer LDAP za pomocą Dockera.
2. Dodałeś dane do LDAP za pomocą pliku LDIF.
3. Odpytujesz LDAP i wykonujesz operacje CRUD w Pythonie.
